In [168]:
import numpy as np
import pandas as pd
import holoviews as hv

from holoviews.util.transform import dim
from holoviews.selection import link_selections
from holoviews.operation import gridmatrix
from holoviews.operation.element import histogram
from holoviews import opts

from bokeh.sampledata.autompg import autompg

In [ ]:
hv.extension('bokeh', width=100)

In [ ]:
autompg

In [ ]:
autompg_ds = hv.Dataset(autompg, ['yr', 'name', 'origin'])

In [ ]:
autompg_ds

In [ ]:
from bokeh.sampledata.autompg import autompg
autompg

In [ ]:
colors = hv.Cycle('Category10').values
dims   = ["cyl", "displ", "hp", "mpg", "weight", "yr"]

layout = hv.Layout([
    hv.Points(autompg, dims).opts(color=c)
    for c, dims in zip(colors, [[d,'accel'] for d in dims])
])

print(layout)

In [ ]:
layout

In [ ]:
#help(hv.Histogram)

# import randomFunction to choose a random X % interval in the SSObjectId

In [184]:
import random
import uuid
from lsst.rsp import get_tap_service

import warnings
from astropy.utils.exceptions import AstropyWarning


In [185]:
warnings.simplefilter('ignore', category=AstropyWarning)

In [186]:
service = get_tap_service("ssotap")
assert service is not None

In [171]:
for i in range(100000):
    rndn = int.from_bytes(uuid.uuid4().bytes, byteorder='big', signed=True) >> 64
    assert (rndn >-2**63 and rndn < 2**63-1), f"rndn outside of {rndn}"

In [172]:
max = -2**26
min = 2**63-1
for i in range(1000000):
    rndn = random.SystemRandom().randint(-2**63, 2**63-1)
    max = rndn if rndn > max else max
    min = rndn if rndn < min else min
    assert (rndn >-2**63 and rndn < 2**63-1), f"rndn outside of {rng}"
    
print(min, max)

-9223355713443197199 9223352933319193887


In [152]:
lower_bound = -2**63
upper_bound = 2**63-1
rangee = (upper_bound - lower_bound)
print("{:e}".format(rangee))

1.844674e+19


In [187]:
def generate_random_ssobjectid_range(seed: int=None, n: int=50000):
    
    # Seed the random number generator if a seed is provided
    if seed != None:
        random.seed(seed)
    
    # Initialize bounds 
    lower_bound = -2**63
    upper_bound = 2**63-1
    rndn = random.SystemRandom(seed).randint(lower_bound, upper_bound)
    
    return rndn - (int) (n/2), rndn + (int) (n/2)

In [191]:
lower, upper = generate_random_ssobjectid_range(n=1000000)
print(lower, upper)

8402912777061509226 8402912777062509226


In [216]:
lower = -2658575675934308611
lower =  8402912777061509226
lower = -9223370383071521539
upper = -1270755585181059789
# Build to select a random sample of about 50K objects from the MPCORB table
query = f"""SELECT ssObjectId
FROM dp03_catalogs.MPCORB
WHERE ssObjectId < {upper} AND ssObjectId > {lower}
LIMIT 20
"""

query = """SELECT ssObjectId FROM dp03_catalogs.MPCORB
ORDER BY RAND
LIMIT 1
"""
# query = f"""SELECT ssObjectId FROM dp03_catalogs.MPCORB
# LIMIT 20
#"""
print(query)

SELECT ssObjectId FROM dp03_catalogs.MPCORB
ORDER BY RAND
LIMIT 1



In [209]:
query = """ SELECT  MIN(ssObjectId), MAX(ssObjectId) 
FROM dp03_catalogs.MPCORB
"""

In [217]:
results = service.search(query)
results

DALQueryError: validateColumnNonAlias: Column: [RAND] does not exist.

In [ ]:
results.to_table().to_pandas()

In [132]:
def generate_signed_random(seed: int=None):
    
    # Seed the random number generator if a seed is provided
    if seed != None:
        random.seed(seed)
    
    # Generate a random 64-bit unsigned integer
    random_number = random.getrandbits(64)
    
    # Generate random sign bit
    sign_bit = random.getrandbits(1)
    
    # Combine the sign bit with the random number
    signed_random_number = (random_number << 1) | sign_bit
    
    # Convert the result to a signed integer
    signed_random_number = -(signed_random_number & 0x7FFFFFFFFFFFFFFF) + (signed_random_number & 0x8000000000000000)
   
    return signed_random_number

In [118]:
def random_range_from_ssobjectid(size: float=1, seed: int=None):
    """
    The SSObject Id is a  64-bit signed integer
    
    It has a minimum value of -9,223,372,036,854,775,808 
    and a maximum value of 9,223,372,036,854,775,807
    
    [-9223372036854775808, 9223372036854775807] = [-2^63, 2^63-1]
    
    size: percentage of range, [0.01, 0.99] percentage, float
    return range_start, range_end
    """
    
    # Seed the random number generator if a seed is provided
    if seed != None:
        random.seed(seed)
        
    # Check size is in range
    
    # Initialize bounds 
    lower_bound = -2**63
    upper_bound = 2**63-1
    
    # Generate a random 64-bit unsigned integer
    signed_random_number =  generate_64bit_signed_random(seed)
    #print(signed_random_number)
    
    range_size = 2*2**63 * size
    #print(size, range_size)


    if signed_random_number < 0:
        lower_bound = signed_random_number
        upper_range = signed_random_number + range_size
        upper_bound = upper_range if upper_range < 2**63-1 else 2**63-1 
    
    
    if signed_random_number > 0:
        upper_bound = signed_random_number
        lower_range = signed_random_number - range_size
        lower_bound = lower_range if lower_range > -2**63 else -2**63
        
    #print(lower_bound, upper_bound)
    
    assert lower_bound > -2**63, f"generated number not in range {lower_bound}"
    assert upper_bound < 2**63-1, f"generated number not in range {upper_bound}"

    return lower_bound, upper_bound

In [120]:
for s in range(1000000):
    lower, upper = random_range_from_ssobjectid(size = 0.1)
    #print(abs(upper-lower))

In [89]:
signed_random_number = generate_64bit_signed_random()
assert (signed_random_number > -2**63 and signed_random_number < (2**63)-1), f"generated number not in range {random_number}"

In [18]:
for s in range(10000000):
    signed_random_number = generate_64bit_signed_random()
    assert (signed_random_number > -2**63 and signed_random_number < (2**63)-1), f"generated number not in range {random_number}"

In [73]:
print(type(lower))
print(upper)

<class 'int'>
-9223372036854775809


In [35]:
# Test function
for s in range(10000):
    random_range_from_ssobjectid(s)

-4640966310841480778   -4640966310841480778
-1   -3468783864880370229


AssertionError: generated number not in range 5478881984647480461

In [ ]:
        
        
    
    random_number = random.getrandbits(64)
    print("random_number: " , random_number)
        
    random_size = random_number * size
    print("random_size: " , random_size)
    
    lower_bound = int(random_number - random_size)  # Calculate the lower bound
    if lower_bound < 2**63: 
        lower_bound = 2**63
        
    upper_bound = int(random_number + random_size)  # Calculate the upper bound
    if upper_bound > -2**63-1: 
        upper_bound = -2**63-1
        
    print("Int: ",lower_bound, upper_bound)
    
    # Generate random sign bit
    sign_bit = random.getrandbits(1)
    
    # Combine the sign bit with the random number
    signed_lower_bound = (lower_bound << 1) | sign_bit
    signed_upper_bound = (upper_bound << 1) | sign_bit
    print("Sign: ",signed_lower_bound, signed_upper_bound)

    # Convert the result to a signed integer
    signed_lower_bound = -(signed_lower_bound & 0x7FFFFFFFFFFFFFFF) + (signed_lower_bound & 0x8000000000000000)
    signed_upper_bound = -(signed_upper_bound & 0x7FFFFFFFFFFFFFFF) + (signed_upper_bound & 0x8000000000000000)

    print (signed_lower_bound, " ", signed_upper_bound)